In [1]:
import pandas as pd
import numpy as np
import seaborn as sns



In [2]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier



In [5]:
df=pd.read_csv('Downloads/DataFrames/Titanic-Dataset.csv')

In [6]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [8]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'
],inplace=True)

In [9]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [10]:
X_train,X_test,y_train,y_test=train_test_split(df.drop(columns=['Survived']),df['Survived'],test_size=0.2,random_state=101)

In [11]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
733,2,male,23.0,0,0,13.00,S
857,1,male,51.0,0,0,26.55,S
81,3,male,29.0,0,0,9.50,S
319,1,female,40.0,1,1,134.50,C
720,2,female,6.0,0,1,33.00,S


In [12]:
#imputation

In [13]:
trf1=ColumnTransformer(
    [('impute_age',SimpleImputer(),[2]),
     ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])
    ],remainder='passthrough')



In [15]:
#OneHotEncoding

In [16]:
trf2= ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse=False,handle_unknown='ignore'),[1,6])
],remainder='passthrough')

In [17]:
#scaling

In [18]:
trf3= ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))
])

In [19]:
#featureScaling



In [20]:
trf4=SelectKBest(score_func=chi2,k=8)

In [21]:
trf5=DecisionTreeClassifier()

In [22]:
pipe = make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [23]:
from sklearn import set_config
set_config(display='diagram')

In [24]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('selectkbest',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x7f825ba00b80>)),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [25]:
y_pred= pipe.predict(X_test)

In [28]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6033519553072626

In [29]:
import pickle

In [30]:
pickle.dump(pipe,open('pipe.pkl','wb'))